# Documentação do Projeto: Integração do Agente de IA com LangSmith

**Autor(a)** Debora da Costa Medeiros

**Contato:** `[debora.medeiros@icomp.ufam.edu.br]`

---

### 1. Descrição Geral

Este documento descreve o processo de integração de um agente de IA baseado em LLM com o LangSmith, com o objetivo de monitorar, depurar e avaliar seu desempenho. O projeto inclui o estudo da documentação oficial do LangSmith, configuração do ambiente, criação de um exemplo funcional e análise de métricas como custo, latência e qualidade.

### 2. Objetivos
- Explorar e entender a documentação oficial do LangSmith.
- Configurar o ambiente para integração.
- Criar um exemplo funcional de agente de IA integrado ao LangSmith.
- Monitorar e capturar métricas relevantes do agente (entradas, saídas, erros, desempenho).
- Realizar testes para avaliar a qualidade dos resultados.

### 3. Configuração do Ambiente
- 3.1 Requisitos
   - Linguagem de Programação: Python (versão 3.12.7)
   - Bibliotecas Necessárias:
   - langchain
   - langsmith
   - openai (ou outra LLM que estiver sendo utilizada)

- 3.2 Instalação das Dependências
   - Execute o seguinte comando para instalar as bibliotecas:

In [ ]:
pip3 install -U langsmith openai

- 3.3 Configuração da API

No terminal configure as seguinte variáveis de ambiente:

In [ ]:
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY="sua_chave_de_api_aqui"
export OPENAI_API_KEY="sua_chave_de_api_aqui"

O **LANGCHAIN_TRACING_V2=true**  ativa o rastreamento global do LangSmith. Com essa configuração ativa, qualquer script que utilize a biblioteca LangChain automaticamente será rastreado, independentemente se for usado ou não explicitamente o wrappers.wrap_openai ou o decorador @traceable no código.

Para conseguir a chave de API do LangChain para LangSmith é preciso acessar o LangSmith em https://www.langchain.com/langsmith e:
- Fazer login com uma conta (pode ser via GitHub ou email).
- Na lateral esquerda procurar por Settings (Configurações).
- Em API Keys, clicar em Create New Key.
- Copiar a chave gerada — ela será usada para autenticar as chamadas na biblioteca LangChain.

Para fins de teste crie o seguinte script python:

In [ ]:
import openai
from langsmith import wrappers, traceable

# Auto-trace LLM calls in-context
client = wrappers.wrap_openai(openai.Client())

@traceable # Auto-trace this function
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

pipeline("Hello, world!")
# Out:  Hello there! How can I assist you today?

### Explicação do script

- `[openai]`: biblioteca oficial para acessar modelos de linguagem da OpenAI, como o GPT.
- `[langsmith.wrappers]`: fornece a funcionalidade para envolver (`[wrap]`) o cliente da OpenAI com suporte ao rastreamento.
- `[langsmith.traceable]`: decorador que permite rastrear automaticamente a execução de funções específicas.

- `[client = wrappers.wrap_openai(openai.Client())]`: A função `[wrappers.wrap_openai]` envolve o cliente da OpenAI (`[openai.Client()]`) com funcionalidades de rastreamento, permitindo que todas as chamadas realizadas por meio dele sejam automaticamente registradas pelo LangSmith.

- O rastreamento captura detalhes como a entrada do usuário, o modelo usado, a saída gerada e o tempo de execução.

- Quando a função `[pipeline]` é executada o decorador `[@traceable]` registra automaticamente a sua execução e coleta informações como:
     - Argumentos de entrada (User Input);
     - Saída da função (Output);
     - Hora de início (End Time)
     - Hora do Fim (End Time)
     - Tempo para o primeiro token (Time to First Token)
     - Status (Sucesso ou Falha)
     - Total tokens
     - Latência

- O `[client.chat.completions.create]` envia uma solicitação ao modelo de linguagem com o prompt fornecido em `[user_input]`
     - `[messages]`: contém uma lista de mensagens no formato necessário para os modelos de chat da OpenAI. Aqui, há apenas uma mensagem com o papel de `["user"]` e o conteúdo fornecido pelo usuário.
     - `[model="gpt-4o-mini"]`: especifica o modelo de linguagem que será usado. Neste caso é o`["gpt-4o-mini"]`.

- `[pipeline("Hello, world!")]` é a chamada da função que envia a mensagem `["Hello, world!]` ao modelo.

- A saída `["Hello there! How can I assist you today?"]` é um exemplo de resposta gerada pelo modelo.

- Como a função está decorada com @traceable, toda a execução é registrada automaticamente.